# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
import os

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "06_Python-APIs/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [23]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
country_code=[]


# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=15)
lngs = np.random.uniform(low=-180.000, high=180.000, size=15)
lat_lngs = zip(lats, lngs)


# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        country_code.append(country)
        
#
       
#print the city count to confirm sufficient count
print(len(cities))
#create data frame to store this information. Columns with additional info can be appended later
weather_df=pd.DataFrame({"City": cities,"Country":country_code})
#created a csv to view list of cities 
weather_df.to_csv("test_cities.csv", index=False)

15


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [26]:
import openweathermapy.core as owm
from api_keys import api_key
import json


settings = {"units": "metric", "appid": api_key}
weather_data = []
#weather_df["Country"]=""
weather_df["Latitude"]=""
weather_df["Longitude"]=""
weather_df["DateTime"]=""
weather_df["CurrentTemp"]=""
weather_df["MaxTemp"]=""
weather_df["WindSpeed"]=""
weather_df["Cloudiness"]=""
weather_df["Humidity"]= ""




for index, row in weather_df.iterrows():
    #want the city name from my df
    city=row['City'].title()
    country=row['Country'].upper()
    time.sleep(1)

    
    #make API request and print processing message
    print(f"Retrieving record {index+1} of {len(cities)}: {city}, {country}.")
    try:
        weather_response=owm.get_current(city,**settings)
    
        #print(weather_response)
        print(f"The weather in {city} is {weather_response['main']['temp']}C with {weather_response['weather'][0]['description']}.")
        
        #weather_df.loc[index, 'Country']=weather_response['sys']['country']
        weather_df.loc[index, 'Latitude'] = weather_response['coord']['lat']
        weather_df.loc[index, 'Longitude'] = weather_response['coord']['lon']
        weather_df.loc[index, 'DateTime']= weather_response['dt']
        weather_df.loc[index, 'CurrentTemp'] = weather_response['main']['temp']
        weather_df.loc[index, 'MaxTemp']= weather_response['main']['temp_max']
        weather_df.loc[index, 'Humidity']=weather_response['main']['humidity']
        weather_df.loc[index, 'WindSpeed'] = weather_response['wind']['speed']
        weather_df.loc[index, 'Cloudiness']= weather_response['clouds']['all']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    except:
        print(f'* No weather data available for this city. ({city}, {country}) *') 
        
    print("------------")
    

Retrieving record 1 of 15: Illoqqortoormiut, GL.
* No weather data available for this city (Illoqqortoormiut, GL) *
------------
Retrieving record 2 of 15: Severo-Kurilsk, RU.
The weather in Severo-Kurilsk is -0.5C with overcast clouds.
------------
Retrieving record 3 of 15: Port Alfred, ZA.
The weather in Port Alfred is 17.22C with broken clouds.
------------
Retrieving record 4 of 15: Fatehabad, IN.
The weather in Fatehabad is 22.06C with clear sky.
------------
Retrieving record 5 of 15: Saint-Philippe, RE.
The weather in Saint-Philippe is -0.47C with light snow.
------------
Retrieving record 6 of 15: Cayenne, GF.
The weather in Cayenne is 26C with broken clouds.
------------
Retrieving record 7 of 15: Bluff, NZ.
The weather in Bluff is 35.16C with clear sky.
------------
Retrieving record 8 of 15: Punta Arenas, CL.
The weather in Punta Arenas is 7.89C with scattered clouds.
------------
Retrieving record 9 of 15: Motygino, RU.
The weather in Motygino is 0.66C with overcast clouds

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [15]:
weather_df.to_csv("weather_data.csv", index=False)

In [16]:
weather_df.head(10)

,City,Country,Latitude,Longitude,DateTime,CurrentTemp,MaxTemp,WindSpeed,Cloudiness,Humidity
0,tsihombe,,,,,,,,,
1,tuatapere,NZ,-46.13,167.69,1552527605,14.44,14.44,1.34,80,62
2,hofn,IS,64.25,-15.21,1552528061,2.61,2.61,11.86,92,100
3,coquimbo,CL,-29.95,-71.34,1552525200,16,16,2.6,90,87
4,umzimvubu,,,,,,,,,
5,saint george,GR,39.45,22.34,1552524600,7,7,1,75,75
6,puerto ayora,EC,-0.74,-90.35,1552527963,25.56,25.56,3.13,0,84
7,port lincoln,AU,-34.72,135.86,1552528151,20.25,20.25,2.96,0,85
8,bredasdorp,ZA,-34.53,20.04,1552527994,17.56,18,2.21,100,88
9,victoria,BN,5.28,115.24,1552525200,27.62,29,0.5,75,74


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot